In [3]:
import numpy as np

In [4]:
def loadDataSet(fileName):
    dataMat = [] ; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat #返回的数据和标签是list

In [8]:
def selectJrand(i,m):
    j = i
    while (j == i):
        j = int(np.random.uniform(0,m))
        #np.random.uniform(low,high,size) 从一个均匀分布[low,high)中随机采样
    return j
# A value is randomly chosen and return as long as it's not equal to the input i

In [9]:
def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [11]:
dataArr,labelArr = loadDataSet('testSet.txt')

In [30]:
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    b = 0
    m,n = dataMatrix.shape
    alphas = np.mat(np.zeros((m,1)))
    iters = 0
    while (iters < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b  
            #f(x)=E a(i)*y(i)<xi,x> + b
            #np.multiply() 对应元素相乘
            Ei = fXi - float(labelMat[i]) #误差
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m) #randomly select second alpha
                fXj = float(np.multiply(alphas,labelMat).T * (dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                if (labelMat[i] != labelMat[j]):
                    L = max(0,alphas[j]-alphas[i])
                    H = min(C,C+alphas[j]-alphas[i])
                else:
                    L = max(0,alphas[j]+alphas[i]-C)
                    H = min(C,alphas[j]+alphas[i])
                if L==H: 
                    print ('L==H')
                    continue
                eta = 2.0*dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                # calculate eta(the similarity of sample i and j) 
                if eta >= 0:
                    print ('eta>=0')
                    continue
                # update alpha[j]
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (np.abs(alphas[j] - alphaJold) < 0.00001):
                    print ('J not moving enough')
                    continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                b1 = b - Ei - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                if (0<alphas[i]) and (C>alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1+b2)/2.0
                alphaPairsChanged += 1
                print ('iter: %d i:%d,pair changed %d' %(iters,i,alphaPairsChanged))
            if (alphaPairsChanged == 0): 
                iters += 1
            else:
                iters = 0
            print('iteration number: %d' % iters)
        return b,alphas      

In [33]:
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)

iter: 0 i:0,pair changed 1
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
L==H
iteration number: 0
J not moving enough
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
J not moving enough
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
L==H
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
L==H
J not moving enough
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
L==H
iteration number: 0
iteration number: 0
iteration number: 0
iteration number: 0
iteration num

In [34]:
b

matrix([[-2.6534237]])

In [35]:
alphas[alphas>0]

matrix([[ 0.11766947,  0.11766947,  0.00404363,  0.00404363]])

In [36]:
np.zeros((5,1))

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])